In [17]:
import numpy as np
import pandas as pd
import tweepy
import os
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
consumerKey = os.getenv("consumerKey")
consumerSecret = os.getenv("consumerSecret")
accessToken = os.getenv("accessToken")
accessTokenSecret = os.getenv("accessTokenSecret")
location = os.getenv("location")

In [19]:
def pulling(location, noOfTweet):
    auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
    auth.set_access_token(accessToken, accessTokenSecret)
    api = tweepy.API(auth)

    keyword = f'vaccine {location}'
    tweets = tweepy.Cursor(api.search, q=keyword).items(noOfTweet)
    print(f'getting data...')
    tweet_list = [[tweet.user.screen_name, tweet.user.followers_count, tweet.user.friends_count, 
                    tweet.retweet_count, tweet.favorite_count, tweet.text] for tweet in tweets]
    tweet_text = pd.DataFrame(data=tweet_list, columns=['user', "followers_count", "friends_count", 
                                                        "retweet_count", "favorite_count", "text"])
    tweet_text.to_csv(f'./dataset/tweet-df-{location}.csv', index=False)

    return tweet_text

In [20]:
noOfTweet = 10000
df = pulling(location, noOfTweet)

getting data...


TweepError: Twitter error response: status code = 429

In [100]:
df = pd.read_csv(f'./dataset/tweet-df-{location}.csv', sep=',', lineterminator='\n')
print(df)

                 user  followers_count  friends_count  retweet_count  \
0     paperteacher121             1107           1359              0   
1           ryan_chua           423705           3308              0   
2        StrawberryNG             1919           1676              0   
3       AlbertMylesAM             2969           4770              0   
4           feedpushr               80              0              0   
...               ...              ...            ...            ...   
1254  JasonLe89224405              454            716            250   
1255      June_yu2020              115            335            250   
1256        trityleti               51            249            250   
1257        redesovir                9             16            250   
1258     heidiwyleung               45             81            250   

      favorite_count                                               text  
0                  0  Hong Kong’s vaccine drive is faltering.

In [101]:
retweeted = []
user = []
user_followers = []
text = []
for index, row in df.text.iteritems():
    if "RT @" in row:
        username = row.split("RT @",1)[1].split(":",1)[-2]
        if " " in username:
            username = username.split(" ",1)[-2]
        retweeted.append(username)
        user.append(df.user[index])
        user_followers.append(df.followers_count[index])
        text.append(df.text[index])
edge = pd.DataFrame(zip(user, user_followers, retweeted, text), columns =['user', 'user_followers', 'retweeted_user', 'text'])
edge = edge.drop(edge[edge.user == edge.retweeted_user].index)
edge['retweeted_user_followers'] = ""
edge['sentiment'] = ""

In [102]:
retweeted_user_list = list(edge.retweeted_user)
retweeted_user_list = list(dict.fromkeys(retweeted_user_list))
# print(retweeted_user_list)

In [103]:
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

retweeted_user_followers_dict = {}

for screen_name in retweeted_user_list:
    # fetching the user
    print(f'fetching {screen_name}...')
    user = api.get_user(screen_name, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
    
    # fetching the followers_count
    followers_count = user.followers_count

    retweeted_user_followers_dict[screen_name] = followers_count

fetching firstdraftnews...
fetching BloombergQuint...
fetching DavidGr07837209...
fetching AlanCane604...
fetching Quicktake...
fetching nikki_miumiu...
fetching tripperhead...
fetching abcnews...
fetching dwnews...
fetching BillyBostickson...
fetching johnrobb...
fetching NikkeiAsia...
fetching nogoodgods...
fetching swissbusiness...
fetching hannahsmumby...
fetching ChinaSunSong...
fetching TIME...
fetching Diplomat_APAC...
fetching next_china...
fetching otandp...
fetching Undergroundsar3...
fetching george_chen...
fetching VamosNadalClay...
fetching bobbiharlow...
fetching Network_Easy...
fetching primroseriordan...
fetching baconmashwbs...
fetching vicciho...
fetching SCMPHongKong...
fetching business...
fetching iainmarlow...
fetching HKU_CMEL...
fetching tommywalkerco...
fetching CanisPundit...
fetching ML03509599...
fetching NavitaSrikant...
fetching gigi_choy...
fetching hongkong_news...
fetching SCMPNews...
fetching paoloigna1...
fetching hildabast...
fetching BrittClennett..

In [104]:
retweeted_user_followers_list = [retweeted_user_followers_dict[i] for i in list(edge.retweeted_user)]
edge['retweeted_user_followers'] = retweeted_user_followers_list
edge.to_csv(f'./dataset/edge-{location}.csv', index=False)